In [1]:
from tqdm import tqdm
import gc
import torch
from torch.nn.functional import softmax
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.metrics import auc
import pandas as pd

In [2]:
# Funcion para hallar la especificidad
# Argumentos:
#      - y_true: vector que contiene los valores de las etiquetas verdaderas
#      - y_pred: vector que contiene los valores de las etiquetas predichas
#

def specificity_score(y_true, y_pred):
    
    # Calculamos la matriz de confusion
    matrix = confusion_matrix(y_true, y_pred)
    
    # Si predicho y verdaderos son iguales
    if matrix.size == 1:
        tn = matrix[0][0]
        fp = 0
    else:
        tn = matrix[0][0]
        fp = matrix[0][1]
        fn = matrix[1][0]
        tp = matrix[1][1]
    
    # Hallamos la especificidad
    specificity = tn / (tn+fp)
    return specificity

In [3]:
# Funcion halla las métricas de especificidad, precision y sensibilidad, para un lote
# Argumentos:
#      - model: contiene el modelo preentrenado
#      - test_loader: lote de imagenes de prueba
#

def build_metrics_(model, test_loader):
    
    # Inicializamos un diccionario para almacenar las métricas
    metric_results = {}

    # Iteramos sobre el conjunto de test
    for i, (images, masks) in tqdm(enumerate(test_loader), total=len(test_loader)):

        # Permuta las dimensiones de las imágenes a (N, C, H, W)
        images = images.permute(0, 3, 1, 2)
        
        # Movemos al device las imagenes y las Ground Truth
        images = images.to(device, dtype=torch.float)
        masks = masks.to(device, dtype=torch.long)
        
        # El modelo realiza la inferencia para las imagenes del lote
        outputs = model(images)
        
        # Aplicamos la funcion softmax para obtener las probabilidad
        outputs = softmax(outputs, dim=1)

        # Convertimos las predicciones y las etiquetas a arrays de numpy
        all_preds = outputs.detach().cpu().numpy()
        all_labels = masks.cpu().numpy()

        # Iteramos sobre los umbrales y las clases
        for threshold in [round(x * 0.1, 1) for x in range(0, 11)]:
            
            for idx_class in range(5):
                
                # Binarizamos las salidas
                preds = (all_preds[:, idx_class, :, :] > threshold).reshape(-1)
                
                # Creamos un vector con las etiquetas reales
                true = (all_labels == idx_class).reshape(-1)

                # Calculamos las métricas
                precision = precision_score(true, preds)
                recall = recall_score(true, preds)  
                specificity = specificity_score(true, preds)

                # Añadimos un diccionario para cada clase
                if idx_class not in metric_results:
                    metric_results[idx_class] = {'thresholds': [], 'specificity': [], 'recall': [], 'precision': []}
                
                # Añadimos los valores de las métricas
                metric_results[idx_class]['thresholds'].append(round(threshold, 2))
                metric_results[idx_class]['specificity'].append(round(specificity, 5))
                metric_results[idx_class]['recall'].append(round(recall, 5))
                metric_results[idx_class]['precision'].append(round(precision, 5))

        # Limpiamos 
        del images, masks, outputs, all_preds, all_labels
        gc.collect()

    return metric_results

In [4]:
# Halla las medias de las metricas para todos los lotes, generando asi la media global
# Argumentos:
#      - metric_results: contiene las métricas para todos los lotes

def mean_metrics(metric_results):
    
    # Diccionario que tiene las metricas medias para cada clase
    mean_metrics = {}

    for idx_class in metric_results.keys():

        # Creamos un array de diccionarios, donde cada uno tiene las metricas para cada clase
        mean_metrics[idx_class] = {'thresholds': [], 'specificity': [], 'recall': [], 'precision': []}

        # Obtenemos las listas de métricas para esta clase
        thresholds = metric_results[idx_class]['thresholds']
        specificity = metric_results[idx_class]['specificity']
        recall = metric_results[idx_class]['recall']
        precision = metric_results[idx_class]['precision']

        # Calculamos la media de las métricas para cada umbral, así agrupamos las metricas por umbral
        for threshold in set(thresholds):
            
            # Creamos una lista vacía para almacenar los índices
            indices = []

            # Recorremos thresholds
            for i in range(len(thresholds)):
                
                # Obtenemos el elemento en la posición 'i'
                x = thresholds[i]
    
                if x == threshold:
                    indices.append(i)
                
            # Conseguimos que indices tenga la metricas agrupada por umbral
            mean_metrics[idx_class]['thresholds'].append(threshold)
            
            # Hallamos las medias para las 3 metricas
            mean_metrics[idx_class]['specificity'].append(np.mean([specificity[i] for i in indices]))
            mean_metrics[idx_class]['recall'].append(np.mean([recall[i] for i in indices]))
            mean_metrics[idx_class]['precision'].append(np.mean([precision[i] for i in indices]))
    
    return mean_metrics

In [5]:
# Printea las métricas globales
# Argumentos
#      - averaged_metrics: contiene las métricas globales
#      - classes: lista con los nombres de las clases

def print_metrics(mean_metrics, classes):
    
    for idx_class in mean_metrics.keys():
        
        # Obtenemos las listas de métricas para esta clase
        thresholds = mean_metrics[idx_class]['thresholds']
        specificity = mean_metrics[idx_class]['specificity']
        recall = mean_metrics[idx_class]['recall']
        precision = mean_metrics[idx_class]['precision']

        # Creamos una lista de tuplas, donde cada tupla contiene el umbral y las métricas correspondientes
        metrics = list(zip(thresholds, specificity, recall, precision))

        # Ordenamos la lista de tuplas por el umbral
        metrics.sort()

        # Desempaquetamos la lista de tuplas ordenada de nuevo en las listas de métricas
        thresholds, specificity, recall, precision = zip(*metrics)

        # Actualizamos las listas de métricas en el diccionario
        mean_metrics[idx_class]['thresholds'] = list(thresholds)
        mean_metrics[idx_class]['specificity'] = list(specificity)
        mean_metrics[idx_class]['recall'] = list(recall)
        mean_metrics[idx_class]['precision'] = list(precision)
    
    
    
    # Crear DataFrames para cada clase
    all_dataframes = {}

    # Generamos un DataFrame para cada clase
    for idx_class, metrics in mean_metrics.items():
        data = {
            'Threshold': metrics['thresholds'],
            'Sensibilidad': metrics['recall'],
            'Especificidad': metrics['specificity'],
            'Precision': metrics['precision']
        }

        # Creamos el DataFrame
        df = pd.DataFrame(data)

        # Redondeamos a 5 decimales
        df = df.round(5) 

        # Almacenamos el DataFrame en el diccionario
        # key = nombre de la clase
        # value = el diccionario
        all_dataframes[classes[idx_class]] = df

    # Acceder a los DataFrames individuales
    for class_name, df in all_dataframes.items():

        # Imprimimos las metricas
        print(f"\Resultados para la clase {class_name}:\n")
        print(df)

In [6]:
# Dibuja las curvas ROC, una para cada clase
# Argumentos
#      - metric_results: contiene las métricas globales
#      - classes: lista con los nombres de las clases

def print_roc_curve(metric_results, classes):
    
    # Accede a los resultados almacenados
    for idx_class, metrics in metric_results.items():

        # Obtenemos las metricas d interes
        thresholds = metrics['thresholds']
        specificity_values = metrics['specificity']
        recall_values = metrics['recall']

        # Calculamos la tasa de falsos positivos (1 - especificidad)
        fpr = [1 - spec for spec in specificity_values]

        # Calculamos el área bajo la curva ROC
        roc_auc = auc(fpr, recall_values)

        plt.figure()

        # Dibujamos la curva roc
        plt.plot(fpr, recall_values, color='orange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        
        # Definimos que los ejes, van de 0 a 1
        plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='-')

        # Etiquetas
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve - Class ' + str(idx_class) + " ~ " + classes[idx_class])

        # Elegimos la posicion de la leyenda
        plt.legend(loc='lower right')
        plt.show()